# Local descriptor

In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import math

In [4]:
def clip_and_count(gradient, threshold):
    """
        I used Liwei Yang 1008229631's flag method to reduce running complexity
    """
    flags = np.zeros(shape=(gradient.shape[0], gradient.shape[1]), dtype=int)
    height = gradient.shape[0]
    width = gradient.shape[1]
    groups = []
    for h in range(height):
        for w in range(width):
            magnitude = np.linalg.norm(gradient[h][w])
            if magnitude >= threshold:
                ratio = np.divide(gradient[h][w][0], gradient[h][w][1],
                                  out=np.zeros_like(gradient[h][w][0]), where=gradient[h][w][1]!=0)
                direction = (np.arctan(ratio) * 180 / math.pi)
                if direction >= 0:
                    group = int((direction + 15) / 25)
                else:
                    group = int((180 - np.abs(direction) + 15) / 25)
                flags[h][w] = group + 1
                if group not in groups:
                    groups.append(group)
                    groups.append(direction)
    flags[flags > 6] = 1
    return flags

In [5]:
def to_box(gradient_magnitude, flags, tau=25):
    add = np.zeros(shape=(tau, tau, 6), dtype=float)
    count = np.zeros(shape=(tau, tau, 6), dtype=int)
    dh = int(flags.shape[0]/tau)
    dw = int(flags.shape[1]/tau)
    for h in range(tau):
        for w in range(tau):
            block = flags[h * dh: h * dh + dh]
            box = block.T[w * dw:w * dw + dw].flatten()

            magnitude_block = gradient_magnitude[h * dh: h * dh + dh]
            magnitude_box = magnitude_block.transpose(1, 0)[strat_w:end_w].transpose(1, 0).flatten()

            for k in range(6):
                indices = np.where(flag_block == (k+1))
                ct = np.take(magnitude_block, indices)
                add[h][w][k] = np.sum(ct)
                count[h][w][k] = np.count_nonzero(box == (k+1))
    return count, add

In [9]:
def normalize(unnormalized, tau=25):
    constant = 0.001
    rslt =np.zeros(shape=(unnormalized.shape[0]-1, unnormalized.shape[1]-1, 6), dtype=float)
    height = tensor.shape[0]
    width = tensor.shape[1]
    dh = int(height/tau)
    dw = int(width/tau)
    for h in range(tau-1):
        for w in range(tau-1):
            block = tensor[h * dh:h * dh + dh  + dh]
            box = block.transpose(1, 0, 2)[w * dw:w * dw + dw + dw].transpose(1, 0, 2).transpose(2, 0, 1)
            raw = np.sum(np.sum(box, axis=1), axis=1)
            normalized = unnormalized_vector / np.sqrt(np.sum(raw * raw) + constant * constant)
            rslt[h][w]  = normalized
    return rslt


In [7]:
def plot_HOG(image, group_box, tau=25, normalize=True):

    x_dir = np.zeros(shape=(6,), dtype=float)
    y_dir = np.zeros(shape=(6,), dtype=float)
    for k in range(6):
        x_dir[k] = np.cos(k * math.pi / 6)
        y_dir[k] = np.sin(k * math.pi / 6)

    dw = int(image.shape[0] / tau)
    dh = int(image.shape[1] / tau)
    fig, ax = plt.subplots()
    ax.imshow(image, cmap='gray')
    for i in range(tau):
        for j in range(tau):
            for k in range(6):
                val = group_box[i][j][k]
                g = group_box[i][j]
                x_ratio = x_dir[k]
                y_ratio = y_dir[k]
                if normalize:
                    magnitude = val / max(np.sum(g), 1)
                else:
                    magnitude = val
                ax.quiver( i * dw + int(dw/2), j * dh + int(dh/2), x_ratio * magnitude, y_ratio * magnitude, width=0.002, scale=2, scale_units='inches', color='r')
                ax.quiver(y_pos, x_pos, -x_direct, -y_direct, width=0.002,
                          scale=2, scale_units='inches', color='r')
    plt.show()
    return None

In [ ]:
threshold = 100
tau = 25
image = cv2.imread('Q3/1.jpg', cv2.IMREAD_GRAYSCALE)
sobelx = np.array(cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3))
sobely = np.array(cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3))
gradient = np.array([sobelx, sobely]).transpose(1, 2, 0)
flags = clip_and_count(gradient=gradient, threshold=threshold)
count, add = to_box(gradient_magnitude=np.sqrt(sobelx * sobelx + sobely * sobely), flags=flags, tau=tau)
normalized = normalize(added)

In [ ]:
plot_HOG(image, flags,normalize=True)

In [ ]:
plot_HOG(image, add, normalize=True)

In [ ]:
plot_HOG(image, normalized, tau=tau-1, normalize=False)
np.savetxt("Normalized_matrix.txt", normalized.flatten())
